In [347]:
import pandas as pd
import numpy as np

Start by considering just the data from 2020. Calls are classfied into several types. What fraction of calls are of the most common type?

In [348]:
calls20 = pd.read_csv('calls2020.csv')  

In [349]:
calls20.head()

,NOPD_Item,Type,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
0,A3472220,22A,AREA CHECK,1K,22A,AREA CHECK,1K,3688756,528696,01/28/2020 01:37:20 AM,...,01/28/2020 01:37:28 AM,01/28/2020 02:25:50 AM,NAT,Necessary Action Taken,N,4G04,Atlantic Ave & Slidell St,70114,4,POINT (-90.04525645 29.94750953)
1,A0000220,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3668710,533007,01/01/2020 12:00:42 AM,...,01/01/2020 12:00:42 AM,01/01/2020 01:37:16 AM,NAT,Necessary Action Taken,Y,2U04,034XX Broadway St,70125,2,POINT (-90.10840522 29.95996774)
2,A2190820,22A,AREA CHECK,1K,22A,AREA CHECK,1K,3682445,530709,01/17/2020 09:18:41 PM,...,01/17/2020 09:18:47 PM,01/17/2020 09:18:54 PM,NAT,Necessary Action Taken,N,8B02,N Peters St & Bienville St,70130,8,POINT (-90.065113 29.95323762)
3,A2874820,21,COMPLAINT OTHER,2A,21,COMPLAINT OTHER,1J,3737616,590067,01/23/2020 10:19:48 AM,...,01/23/2020 10:31:11 AM,01/23/2020 10:34:35 AM,GOA,GONE ON ARRIVAL,N,7L08,I-10 E,70129,7,POINT (-89.88854843 30.11465463)
4,A2029120,34S,AGGRAVATED BATTERY BY SHOOTING,2C,34S,AGGRAVATED BATTERY BY SHOOTING,2C,3696210,551411,01/16/2020 05:09:05 PM,...,01/16/2020 05:16:07 PM,01/16/2020 10:49:37 PM,RTF,REPORT TO FOLLOW,N,7A01,Chef Menteur Hwy & Downman Rd,70126,7,POINT (-90.02090137 30.00973449)


In [350]:
calls20.isnull().sum()

NOPD_Item              0
Type                   0
TypeText               0
Priority               0
InitialType            0
InitialTypeText        0
InitialPriority        0
MapX                   0
MapY                   0
TimeCreate             0
TimeDispatch       26300
TimeArrive         54199
TimeClosed             0
Disposition            0
DispositionText        0
SelfInitiated          0
Beat                   0
BLOCK_ADDRESS          1
Zip                    0
PoliceDistrict         0
Location               0
dtype: int64

In [352]:
typ = calls20.groupby('Type')

In [355]:
calls20.shape

(432892, 21)

In [356]:
typ.size().max() / 432892

0.23926753093150255

Now compare to the data from 2016. Find the call type that displayed the largest percentage decrease in volume between 2016 and 2020. What is the fraction of the 2016 volume that this decrease represents? The answer should be between 0 and 1. (Note that the name of the type column changes over time. You can use the TypeText column, which remains constant, to determine the call type.)

In [359]:
T16 = calls16.groupby('Type').size().to_frame('size16')

In [360]:
T16

,size16
Type,
100,8391
100F,7
100I,816
100X,11
101,1
...,...
97,1
98,521
99,3948


In [361]:
T20 = calls20.groupby('Type').size().to_frame('size20')

In [362]:
T20

,size20
Type,
100,6715
100C,18
100F,5
100I,1125
100X,13
...,...
GATHER,5293
NOPD,661
TOW,5433


In [363]:
merged = pd.merge(T16, T20, on='Type', how='outer')
merged = merged.dropna(subset=['size16'])
merged = merged.fillna(0)
merged

,size16,size20
Type,,
100,8391.0,6715.0
100F,7.0,5.0
100I,816.0,1125.0
100X,11.0,13.0
101,1.0,1.0
...,...,...
97,1.0,0.0
98,521.0,226.0
99,3948.0,3372.0


In [364]:
sum16 = merged['size16'].sum()
sum20 = T20['size20'].sum()

In [365]:
merged['fraction16'] = merged['size16'] / sum16
merged['fraction20'] = merged['size20'] / sum20

In [366]:
merged['decrease'] = merged['fraction16'] - merged['fraction20']
merged

,size16,size20,fraction16,fraction20,decrease
Type,,,,,
100,8391.0,6715.0,0.020767,0.015512,5.254663e-03
100F,7.0,5.0,0.000017,0.000012,5.773849e-06
100I,816.0,1125.0,0.002019,0.002599,-5.793086e-04
100X,11.0,13.0,0.000027,0.000030,-2.807040e-06
101,1.0,1.0,0.000002,0.000002,1.648227e-07
...,...,...,...,...,...
97,1.0,0.0,0.000002,0.000000,2.474868e-06
98,521.0,226.0,0.001289,0.000522,7.673359e-04
99,3948.0,3372.0,0.009771,0.007789,1.981306e-03


In [367]:
merged['decrease'].max()

0.10254235362638953

In [368]:
merged[merged['decrease'] == 0.10254235362638953 ]

,size16,size20,fraction16,fraction20,decrease
Type,,,,,
21,99001.0,61675.0,0.245014,0.142472,0.102542


In [369]:
decrease = 99001.0 - 61675.0

In [370]:
decrease / sum16

0.09237691245402933

In [64]:
import os, glob
import pandas as pd
import csv
import datetime 

path = "/Users/Mahsa/dataScienceProject/section2"

In [3]:
calls16 = pd.read_csv('calls2016.csv')  

In [100]:
renamed_df=calls19.rename(columns={'TimeArrival':'TimeArrive'})
renamed_df.to_csv("/Users/Mahsa/dataScienceProject/section2/calls2019.csv", index=None)

In [110]:
renamed_df=calls18.rename(columns={'Type_':'Type'})
renamed_df.to_csv("/Users/Mahsa/dataScienceProject/section2/calls2018.csv", index=None)

In [116]:
renamed_df=calls17.rename(columns={'Type_':'Type'})
renamed_df.to_csv("/Users/Mahsa/dataScienceProject/section2/calls2017.csv", index=None)

In [119]:
renamed_df=calls16.rename(columns={'Type_':'Type'})
renamed_df.to_csv("/Users/Mahsa/dataScienceProject/section2/calls2016.csv", index=None)

In [68]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

In [69]:
#combine all files in the list
comb = pd.concat([pd.read_csv(f) for f in all_filenames ])

/Users/Mahsa/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [70]:
comb

,NOPD_Item,Type,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
0,D0808216,24,MEDICAL,2B,21,COMPLAINT OTHER,2A,37369000,3513814,04/07/2016 09:24:16 PM,...,04/07/2016 09:37:10 PM,04/07/2016 09:47:44 PM,NAT,Necessary Action Taken,N,8F02,Dauphine St & Pauger St,70116,8,"(0.0, 0.0)"
1,D0812216,18,TRAFFIC INCIDENT,1H,18,TRAFFIC INCIDENT,1H,37369000,3513814,04/07/2016 09:56:49 PM,...,04/07/2016 09:56:49 PM,04/07/2016 09:56:54 PM,NAT,Necessary Action Taken,Y,NaN,Fou & Nas,NaN,0,"(0.0, 0.0)"
2,D0812716,103M,MENTAL PATIENT,2A,103M,MENTAL PATIENT,2A,37369000,3513814,04/07/2016 10:00:09 PM,...,04/07/2016 10:12:58 PM,04/07/2016 10:51:50 PM,GOA,GONE ON ARRIVAL,N,7I13,101XX Curran Blvd,70127,7,"(0.0, 0.0)"
3,I3355216,21,COMPLAINT OTHER,1H,21,COMPLAINT OTHER,1H,0,0,09/30/2016 02:06:54 AM,...,09/30/2016 02:06:54 AM,09/30/2016 04:09:56 AM,NAT,Necessary Action Taken,Y,8C02,003XX Royal St,NaN,0,NaN
4,D0816216,24,MEDICAL,2B,24,MEDICAL,2B,37369000,3513814,04/07/2016 10:48:36 PM,...,04/07/2016 10:48:36 PM,04/07/2016 11:14:57 PM,NAT,Necessary Action Taken,Y,8D06,007XX Bourbon St,70116,8,"(0.0, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460869,L4042618,103M,MENTAL PATIENT,2C,107,SUSPICIOUS PERSON,2C,37369000,3513814,12/31/2018 10:00:40 PM,...,12/31/2018 10:11:09 PM,12/31/2018 11:15:04 PM,NAT,Necessary Action Taken,N,6E04,Us 90B E & Camp St,70130,6,"(0.0, 0.0)"
460870,L4057018,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3678680,532533,12/31/2018 11:43:08 PM,...,12/31/2018 11:43:08 PM,01/01/2019 12:06:43 AM,NAT,Necessary Action Taken,Y,8J04,Canal St & S Robertson St,70112,8,"(29.95836768, -90.07693818)"
460871,L4057818,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3681160,531230,12/31/2018 11:46:23 PM,...,12/31/2018 11:46:23 PM,12/31/2018 11:47:06 PM,NAT,Necessary Action Taken,Y,8D01,Iberville St & Bourbon St,NaN,8,"(29.95470912, -90.06915346)"
460872,L4059118,94F,FIREWORKS,1A,94F,FIREWORKS,2J,3660808,529688,12/31/2018 11:52:45 PM,...,01/01/2019 12:29:55 AM,01/01/2019 12:39:12 AM,GOA,GONE ON ARRIVAL,N,2K01,012XX Leonidas St,70118,2,"(29.95107305, -90.13346863)"


In [71]:
comb.isnull().sum()

NOPD_Item               0
Type                    0
TypeText                0
Priority                0
InitialType             0
InitialTypeText         0
InitialPriority         0
MapX                    0
MapY                    0
TimeCreate              0
TimeDispatch       348389
TimeArrive         365063
TimeClosed              0
Disposition            14
DispositionText        14
SelfInitiated           0
Beat                 3275
BLOCK_ADDRESS          50
Zip                 21462
PoliceDistrict          0
Location             2950
dtype: int64

In [19]:
#comb['TimeClosed'] = pd.to_datetime(comb['TimeClosed'])

In [20]:
#comb['yearClosed'] = pd.DatetimeIndex(comb['TimeClosed']).year

In [30]:
#comb['monthClosed'] = pd.DatetimeIndex(comb['TimeClosed']).month

In [72]:
#comb

In [73]:
#comb[comb.duplicated('NOPD_Item')]

In [62]:
#df = comb[comb.duplicated(['NOPD_Item']) & ~(comb.duplicated('TimeCreate'))]

In [74]:
#df

In [75]:
#df[df.duplicated('yearClosed')]

Work out the average (mean) response time in each district. What is the difference between the average response times of the districts with the longest and shortest times?

In [265]:
comb = comb.dropna(subset=['TimeArrive'])

In [266]:
comb['TimeArrive'] = pd.to_datetime(comb['TimeArrive'])

<ipython-input-266-c4ebdae203b4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['TimeArrive'] = pd.to_datetime(comb['TimeArrive'])


In [267]:
comb['TimeCreate'] = pd.to_datetime(comb['TimeCreate'])

<ipython-input-267-05b9985af48a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['TimeCreate'] = pd.to_datetime(comb['TimeCreate'])


In [268]:
comb = comb[comb['TimeArrive'] >= comb['TimeCreate']]

In [269]:
comb.isnull().sum()

NOPD_Item               0
Type                    0
TypeText                0
Priority                0
InitialType             0
InitialTypeText         0
InitialPriority         0
MapX                    0
MapY                    0
TimeCreate              0
TimeDispatch       190644
TimeArrive              0
TimeClosed              0
Disposition             7
DispositionText         7
SelfInitiated           0
Beat                 3223
BLOCK_ADDRESS          43
Zip                 20448
PoliceDistrict          0
Location             2655
dtype: int64

In [271]:
comb = comb.dropna(subset=['TimeDispatch'])
comb['TimeDispatch'] = comb['TimeDispatch'].apply(pd.Timestamp)

In [272]:
comb['TimeArrive'] = comb['TimeArrive'].apply(pd.Timestamp)
#df['time'] = df['time'].apply(pd.Timestamp)

In [244]:
comb['TimeCreate'] = comb['TimeCreate'].apply(pd.Timestamp)

<ipython-input-244-61da2606afa3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['TimeCreate'] = comb['TimeCreate'].apply(pd.Timestamp)


In [303]:
comb['ResponseTime'] = comb['TimeArrive'] - comb['TimeDispatch']

In [304]:
import numpy as np
comb['ResponseTime'] = comb['ResponseTime'] / np.timedelta64(1, 's')

In [305]:
comb

,NOPD_Item,Type,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,ResponseTime,DispResponse
0,D0808216,24,MEDICAL,2B,21,COMPLAINT OTHER,2A,37369000,3513814,2016-04-07 21:24:16,...,NAT,Necessary Action Taken,N,8F02,Dauphine St & Pauger St,70116,8,"(0.0, 0.0)",641.0,641.0
2,D0812716,103M,MENTAL PATIENT,2A,103M,MENTAL PATIENT,2A,37369000,3513814,2016-04-07 22:00:09,...,GOA,GONE ON ARRIVAL,N,7I13,101XX Curran Blvd,70127,7,"(0.0, 0.0)",583.0,583.0
5,I3357216,29S,SUICIDE,1B,29S,SUICIDE,2A,37369000,3513814,2016-09-30 03:13:02,...,RTF,REPORT TO FOLLOW,N,6B02,019XX Saint Thomas St,70130,6,"(0.0, 0.0)",128.0,128.0
6,I3359916,103M,MENTAL PATIENT,2A,107,SUSPICIOUS PERSON,2A,37369000,3513814,2016-09-30 04:15:14,...,NAT,Necessary Action Taken,N,6K01,033XX Toledano St,70125,6,"(0.0, 0.0)",112.0,112.0
7,D0821716,103M,MENTAL PATIENT,1A,103M,MENTAL PATIENT,1A,37369000,3513814,2016-04-07 23:54:10,...,GOA,GONE ON ARRIVAL,N,4D01,030XX Kabel Dr,70131,4,"(0.0, 0.0)",283.0,283.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1864095,L4047018,20,AUTO ACCIDENT,1E,20,AUTO ACCIDENT,1E,3678265,530150,2018-12-31 22:38:12,...,RTF,REPORT TO FOLLOW,N,8J02,La Salle St & Poydras St,70112,8,"(29.95182765, -90.07833097)",334.0,334.0
1864096,L4042618,103M,MENTAL PATIENT,2C,107,SUSPICIOUS PERSON,2C,37369000,3513814,2018-12-31 22:00:40,...,NAT,Necessary Action Taken,N,6E04,Us 90B E & Camp St,70130,6,"(0.0, 0.0)",529.0,529.0
1864097,L4057018,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3678680,532533,2018-12-31 23:43:08,...,NAT,Necessary Action Taken,Y,8J04,Canal St & S Robertson St,70112,8,"(29.95836768, -90.07693818)",0.0,0.0
1864098,L4057818,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3681160,531230,2018-12-31 23:46:23,...,NAT,Necessary Action Taken,Y,8D01,Iberville St & Bourbon St,NaN,8,"(29.95470912, -90.06915346)",0.0,0.0


In [306]:
grouped = comb.groupby('PoliceDistrict')['ResponseTime'].mean()
from pandas import DataFrame
#df = DataFrame (your_list,columns=['Column_Name'])
group =DataFrame( list(grouped),columns=['responseTime'])
group

,responseTime
0,250.127025
1,284.346935
2,345.093565
3,321.858622
4,360.211683
5,399.929897
6,316.955992
7,620.580126
8,310.113356


In [307]:
longest = comb.groupby('PoliceDistrict')['ResponseTime'].mean().max()

In [308]:
longest

620.5801256053195

In [309]:
shortest = comb.groupby('PoliceDistrict')['ResponseTime'].mean().min()

In [310]:
shortest

250.12702538774803

In [311]:
longest - shortest

370.45310021757143

Some calls result in an officer being dispatched to the scene, and some log an arrival time. What is the median response time (dispatch to arrival), in seconds, considering only valid (i.e. non-negative) times?

In [296]:
comb['TimeDispatch'] = comb['TimeDispatch'].apply(pd.Timestamp)

<ipython-input-296-395607a8eeaa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['TimeDispatch'] = comb['TimeDispatch'].apply(pd.Timestamp)


In [297]:
comb = comb[comb['TimeArrive'] >= comb['TimeDispatch']]

In [298]:
comb['DispResponse'] = comb['TimeArrive'] - comb['TimeDispatch']

In [300]:
comb['DispResponse'] = comb['DispResponse'] / np.timedelta64(1, 's')

In [302]:
comb['DispResponse'].median()

145.0

We can define surprising event types as those that occur more often in a district than they do over the whole city. What is the largest ratio of the conditional probability of an event type given a district to the unconditional probability of that event type? Consider only events types which have more than 100 events. Note that some events have their locations anonymized and are reported as being in district "0". These should be ignored.

In [147]:
df = comb[['PoliceDistrict', 'Type']]

In [148]:
df.reset_index(drop = True, inplace = True)

In [149]:
df = df[df['PoliceDistrict'] != 0]
df.reset_index(drop = True, inplace = True)
df

,PoliceDistrict,Type
0,8,24
1,7,103M
2,8,24
3,6,29S
4,6,103M
...,...,...
2214792,6,103M
2214793,8,21
2214794,8,21
2214795,2,94F


Unconditional Probability for each event type:

In [150]:
df1 = df.groupby('Type').size().to_frame('TypeTotalSize')
df1.reset_index(inplace = True)
df1

,Type,TypeTotalSize
0,100,39971
1,100C,63
2,100F,21
3,100I,4804
4,100X,55
...,...,...
181,NOPD,2710
182,TOW,23607
183,TS,17794
184,UPASS,83


In [151]:
df1['unconditionalProb'] = df1['TypeTotalSize'] / df1['TypeTotalSize'].sum()

In [152]:
df1 = df1[df1['TypeTotalSize'] > 100]
df1

,Type,TypeTotalSize,unconditionalProb
0,100,39971,0.018047
3,100I,4804,0.002169
6,102,422,0.000191
7,1028NP,357,0.000161
8,1028P,152,0.000069
...,...,...,...
180,GATHER,5683,0.002566
181,NOPD,2710,0.001224
182,TOW,23607,0.010659
183,TS,17794,0.008034


In [153]:
df = df.groupby(['PoliceDistrict', 'Type']).size().to_frame('TypeSize')

In [154]:
for i in range(1,9):
    df.loc[i, 'sumDist'] = df.loc[i, 'TypeSize'].sum()


In [155]:
df['conditionalProb'] = df['TypeSize'] / df['sumDist']

In [156]:
df.reset_index(inplace = True)
df

,PoliceDistrict,Type,TypeSize,sumDist,conditionalProb
0,1,100,5767,305563.0,0.018873
1,1,100C,17,305563.0,0.000056
2,1,100F,3,305563.0,0.000010
3,1,100I,741,305563.0,0.002425
4,1,100X,8,305563.0,0.000026
...,...,...,...,...,...
1334,8,NOPD,405,288646.0,0.001403
1335,8,TOW,9640,288646.0,0.033397
1336,8,TS,3005,288646.0,0.010411
1337,8,UPASS,1,288646.0,0.000003


In [157]:
df1

,Type,TypeTotalSize,unconditionalProb
0,100,39971,0.018047
3,100I,4804,0.002169
6,102,422,0.000191
7,1028NP,357,0.000161
8,1028P,152,0.000069
...,...,...,...
180,GATHER,5683,0.002566
181,NOPD,2710,0.001224
182,TOW,23607,0.010659
183,TS,17794,0.008034


In [158]:
merged = pd.merge(df, df1, on='Type')

In [159]:
merged

,PoliceDistrict,Type,TypeSize,sumDist,conditionalProb,TypeTotalSize,unconditionalProb
0,1,100,5767,305563.0,0.018873,39971,0.018047
1,2,100,4350,235267.0,0.018490,39971,0.018047
2,3,100,6384,381985.0,0.016713,39971,0.018047
3,4,100,2669,207336.0,0.012873,39971,0.018047
4,5,100,4840,231835.0,0.020877,39971,0.018047
...,...,...,...,...,...,...,...
1052,7,21U,1,292529.0,0.000003,287,0.000130
1053,8,21U,216,288646.0,0.000748,287,0.000130
1054,3,WALK,439,381985.0,0.001149,441,0.000199
1055,4,WALK,1,207336.0,0.000005,441,0.000199


In [161]:
merged['ratio'] = merged['conditionalProb'] / merged['unconditionalProb']

In [162]:
merged

,PoliceDistrict,Type,TypeSize,sumDist,conditionalProb,TypeTotalSize,unconditionalProb,ratio
0,1,100,5767,305563.0,0.018873,39971,0.018047,1.045775
1,2,100,4350,235267.0,0.018490,39971,0.018047,1.024512
2,3,100,6384,381985.0,0.016713,39971,0.018047,0.926052
3,4,100,2669,207336.0,0.012873,39971,0.018047,0.713284
4,5,100,4840,231835.0,0.020877,39971,0.018047,1.156792
...,...,...,...,...,...,...,...,...
1052,7,21U,1,292529.0,0.000003,287,0.000130,0.026381
1053,8,21U,216,288646.0,0.000748,287,0.000130,5.774844
1054,3,WALK,439,381985.0,0.001149,441,0.000199,5.771830
1055,4,WALK,1,207336.0,0.000005,441,0.000199,0.024223


In [163]:
merged['ratio'].max()

7.047187196886937

Work out the average response time for each month. Make an ordinary least-squares fit to the response time against month. What is the slope of this line, in seconds per year?

In [165]:
comb.reset_index(drop=True, inplace = True)
comb

,NOPD_Item,Type,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeArrive,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location
0,D0808216,24,MEDICAL,2B,21,COMPLAINT OTHER,2A,37369000,3513814,04/07/2016 09:24:16 PM,...,04/07/2016 09:37:10 PM,04/07/2016 09:47:44 PM,NAT,Necessary Action Taken,N,8F02,Dauphine St & Pauger St,70116,8,"(0.0, 0.0)"
1,D0812216,18,TRAFFIC INCIDENT,1H,18,TRAFFIC INCIDENT,1H,37369000,3513814,04/07/2016 09:56:49 PM,...,04/07/2016 09:56:49 PM,04/07/2016 09:56:54 PM,NAT,Necessary Action Taken,Y,NaN,Fou & Nas,NaN,0,"(0.0, 0.0)"
2,D0812716,103M,MENTAL PATIENT,2A,103M,MENTAL PATIENT,2A,37369000,3513814,04/07/2016 10:00:09 PM,...,04/07/2016 10:12:58 PM,04/07/2016 10:51:50 PM,GOA,GONE ON ARRIVAL,N,7I13,101XX Curran Blvd,70127,7,"(0.0, 0.0)"
3,I3355216,21,COMPLAINT OTHER,1H,21,COMPLAINT OTHER,1H,0,0,09/30/2016 02:06:54 AM,...,09/30/2016 02:06:54 AM,09/30/2016 04:09:56 AM,NAT,Necessary Action Taken,Y,8C02,003XX Royal St,NaN,0,NaN
4,D0816216,24,MEDICAL,2B,24,MEDICAL,2B,37369000,3513814,04/07/2016 10:48:36 PM,...,04/07/2016 10:48:36 PM,04/07/2016 11:14:57 PM,NAT,Necessary Action Taken,Y,8D06,007XX Bourbon St,70116,8,"(0.0, 0.0)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229296,L4042618,103M,MENTAL PATIENT,2C,107,SUSPICIOUS PERSON,2C,37369000,3513814,12/31/2018 10:00:40 PM,...,12/31/2018 10:11:09 PM,12/31/2018 11:15:04 PM,NAT,Necessary Action Taken,N,6E04,Us 90B E & Camp St,70130,6,"(0.0, 0.0)"
2229297,L4057018,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3678680,532533,12/31/2018 11:43:08 PM,...,12/31/2018 11:43:08 PM,01/01/2019 12:06:43 AM,NAT,Necessary Action Taken,Y,8J04,Canal St & S Robertson St,70112,8,"(29.95836768, -90.07693818)"
2229298,L4057818,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3681160,531230,12/31/2018 11:46:23 PM,...,12/31/2018 11:46:23 PM,12/31/2018 11:47:06 PM,NAT,Necessary Action Taken,Y,8D01,Iberville St & Bourbon St,NaN,8,"(29.95470912, -90.06915346)"
2229299,L4059118,94F,FIREWORKS,1A,94F,FIREWORKS,2J,3660808,529688,12/31/2018 11:52:45 PM,...,01/01/2019 12:29:55 AM,01/01/2019 12:39:12 AM,GOA,GONE ON ARRIVAL,N,2K01,012XX Leonidas St,70118,2,"(29.95107305, -90.13346863)"


In [166]:
comb.isnull().sum()

NOPD_Item               0
Type                    0
TypeText                0
Priority                0
InitialType             0
InitialTypeText         0
InitialPriority         0
MapX                    0
MapY                    0
TimeCreate              0
TimeDispatch       348389
TimeArrive         365063
TimeClosed              0
Disposition            14
DispositionText        14
SelfInitiated           0
Beat                 3275
BLOCK_ADDRESS          50
Zip                 21462
PoliceDistrict          0
Location             2950
dtype: int64

In [167]:
comb = comb.dropna(subset=['TimeArrive', 'TimeDispatch'])

In [169]:
comb.isnull().sum()

NOPD_Item              0
Type                   0
TypeText               0
Priority               0
InitialType            0
InitialTypeText        0
InitialPriority        0
MapX                   0
MapY                   0
TimeCreate             0
TimeDispatch           0
TimeArrive             0
TimeClosed             0
Disposition            5
DispositionText        5
SelfInitiated          0
Beat                2906
BLOCK_ADDRESS         37
Zip                12956
PoliceDistrict         0
Location            1754
dtype: int64

In [170]:
comb['TimeArrive'] = pd.to_datetime(comb['TimeArrive'])
comb['TimeDispatch'] = pd.to_datetime(comb['TimeDispatch'])

<ipython-input-170-e16be6844cb9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['TimeArrive'] = pd.to_datetime(comb['TimeArrive'])
<ipython-input-170-e16be6844cb9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['TimeDispatch'] = pd.to_datetime(comb['TimeDispatch'])


In [171]:
comb['TimeCreate'] = pd.to_datetime(comb['TimeCreate'])

<ipython-input-171-05b9985af48a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['TimeCreate'] = pd.to_datetime(comb['TimeCreate'])


In [173]:
comb['month_year'] = comb['TimeCreate'].dt.to_period('M')

<ipython-input-173-736cfcc73a33>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comb['month_year'] = comb['TimeCreate'].dt.to_period('M')


In [174]:
comb

,NOPD_Item,Type,TypeText,Priority,InitialType,InitialTypeText,InitialPriority,MapX,MapY,TimeCreate,...,TimeClosed,Disposition,DispositionText,SelfInitiated,Beat,BLOCK_ADDRESS,Zip,PoliceDistrict,Location,month_year
0,D0808216,24,MEDICAL,2B,21,COMPLAINT OTHER,2A,37369000,3513814,2016-04-07 21:24:16,...,04/07/2016 09:47:44 PM,NAT,Necessary Action Taken,N,8F02,Dauphine St & Pauger St,70116,8,"(0.0, 0.0)",2016-04
2,D0812716,103M,MENTAL PATIENT,2A,103M,MENTAL PATIENT,2A,37369000,3513814,2016-04-07 22:00:09,...,04/07/2016 10:51:50 PM,GOA,GONE ON ARRIVAL,N,7I13,101XX Curran Blvd,70127,7,"(0.0, 0.0)",2016-04
3,I3355216,21,COMPLAINT OTHER,1H,21,COMPLAINT OTHER,1H,0,0,2016-09-30 02:06:54,...,09/30/2016 04:09:56 AM,NAT,Necessary Action Taken,Y,8C02,003XX Royal St,NaN,0,NaN,2016-09
5,I3357216,29S,SUICIDE,1B,29S,SUICIDE,2A,37369000,3513814,2016-09-30 03:13:02,...,09/30/2016 03:56:44 AM,RTF,REPORT TO FOLLOW,N,6B02,019XX Saint Thomas St,70130,6,"(0.0, 0.0)",2016-09
6,I3359916,103M,MENTAL PATIENT,2A,107,SUSPICIOUS PERSON,2A,37369000,3513814,2016-09-30 04:15:14,...,09/30/2016 06:13:05 AM,NAT,Necessary Action Taken,N,6K01,033XX Toledano St,70125,6,"(0.0, 0.0)",2016-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2229295,L4047018,20,AUTO ACCIDENT,1E,20,AUTO ACCIDENT,1E,3678265,530150,2018-12-31 22:38:12,...,01/01/2019 12:58:02 AM,RTF,REPORT TO FOLLOW,N,8J02,La Salle St & Poydras St,70112,8,"(29.95182765, -90.07833097)",2018-12
2229296,L4042618,103M,MENTAL PATIENT,2C,107,SUSPICIOUS PERSON,2C,37369000,3513814,2018-12-31 22:00:40,...,12/31/2018 11:15:04 PM,NAT,Necessary Action Taken,N,6E04,Us 90B E & Camp St,70130,6,"(0.0, 0.0)",2018-12
2229297,L4057018,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3678680,532533,2018-12-31 23:43:08,...,01/01/2019 12:06:43 AM,NAT,Necessary Action Taken,Y,8J04,Canal St & S Robertson St,70112,8,"(29.95836768, -90.07693818)",2018-12
2229298,L4057818,21,COMPLAINT OTHER,1J,21,COMPLAINT OTHER,1J,3681160,531230,2018-12-31 23:46:23,...,12/31/2018 11:47:06 PM,NAT,Necessary Action Taken,Y,8D01,Iberville St & Bourbon St,NaN,8,"(29.95470912, -90.06915346)",2018-12


In [207]:
dfR = comb[['TimeDispatch', 'TimeArrive', 'month_year']]

In [208]:
dfR

,TimeDispatch,TimeArrive,month_year
0,2016-04-07 21:26:29,2016-04-07 21:37:10,2016-04
2,2016-04-07 22:03:15,2016-04-07 22:12:58,2016-04
3,2016-09-30 02:07:09,2016-09-30 02:06:54,2016-09
5,2016-09-30 03:15:13,2016-09-30 03:17:21,2016-09
6,2016-09-30 04:18:21,2016-09-30 04:20:13,2016-09
...,...,...,...
2229295,2018-12-31 23:39:34,2018-12-31 23:45:08,2018-12
2229296,2018-12-31 22:02:20,2018-12-31 22:11:09,2018-12
2229297,2018-12-31 23:43:08,2018-12-31 23:43:08,2018-12
2229298,2018-12-31 23:46:23,2018-12-31 23:46:23,2018-12


In [209]:
dfR.reset_index(drop= True, inplace = True)

In [210]:
dfR

,TimeDispatch,TimeArrive,month_year
0,2016-04-07 21:26:29,2016-04-07 21:37:10,2016-04
1,2016-04-07 22:03:15,2016-04-07 22:12:58,2016-04
2,2016-09-30 02:07:09,2016-09-30 02:06:54,2016-09
3,2016-09-30 03:15:13,2016-09-30 03:17:21,2016-09
4,2016-09-30 04:18:21,2016-09-30 04:20:13,2016-09
...,...,...,...
1673585,2018-12-31 23:39:34,2018-12-31 23:45:08,2018-12
1673586,2018-12-31 22:02:20,2018-12-31 22:11:09,2018-12
1673587,2018-12-31 23:43:08,2018-12-31 23:43:08,2018-12
1673588,2018-12-31 23:46:23,2018-12-31 23:46:23,2018-12


In [214]:
dfR = dfR[dfR['TimeArrive'] >= dfR['TimeDispatch']]
dfR.reset_index(drop=True, inplace=True)

In [215]:
dfR['ResponseTime'] = dfR['TimeArrive'] - dfR['TimeDispatch']

In [216]:
dfR['ResponseTime'] = dfR['ResponseTime'] / np.timedelta64(1, 's')

In [218]:
dfR = dfR.drop(['TimeDispatch', 'TimeArrive'], axis=1)

In [219]:
dfR

,month_year,ResponseTime
0,2016-04,641.0
1,2016-04,583.0
2,2016-09,128.0
3,2016-09,112.0
4,2016-04,283.0
...,...,...
1625406,2018-12,334.0
1625407,2018-12,529.0
1625408,2018-12,0.0
1625409,2018-12,0.0


In [220]:
dfR = dfR.groupby('month_year').mean()

In [221]:
dfR

,ResponseTime
month_year,
2016-01,493.212583
2016-02,497.432219
2016-03,489.120220
2016-04,489.058525
2016-05,447.447864
2016-06,494.811126
2016-07,494.470925
2016-08,495.381155
2016-09,517.596893


In [371]:
f = dfR.reset_index()
f

,month_year,ResponseTime
0,2016-01,493.212583
1,2016-02,497.432219
2,2016-03,489.120220
3,2016-04,489.058525
4,2016-05,447.447864
5,2016-06,494.811126
6,2016-07,494.470925
7,2016-08,495.381155
8,2016-09,517.596893
9,2016-10,485.058208


In [372]:
f["month_year"]= f["month_year"].dt.strftime('%Y-%m')

In [373]:
f['month_year'] = pd.to_datetime(f['month_year'])    

In [374]:
f['date_delta'] = (f['month_year'] - f['month_year'].min())  / np.timedelta64(1,'M')


In [375]:
f

,month_year,ResponseTime,date_delta
0,2016-01-01,493.212583,0.000000
1,2016-02-01,497.432219,1.018501
2,2016-03-01,489.120220,1.971293
3,2016-04-01,489.058525,2.989794
4,2016-05-01,447.447864,3.975441
5,2016-06-01,494.811126,4.993942
6,2016-07-01,494.470925,5.979589
7,2016-08-01,495.381155,6.998090
8,2016-09-01,517.596893,8.016592
9,2016-10-01,485.058208,9.002238


In [376]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.regression.linear_model import OLS

result = sm.OLS(f['ResponseTime'], f['date_delta']).fit()
# result = sm.ols(formula = 'ResponseTime ~ date_delta', data = f).fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:           ResponseTime   R-squared (uncentered):                   0.618
Model:                            OLS   Adj. R-squared (uncentered):              0.611
Method:                 Least Squares   F-statistic:                              95.39
Date:                Fri, 09 Apr 2021   Prob (F-statistic):                    6.18e-14
Time:                        18:49:54   Log-Likelihood:                         -414.83
No. Observations:                  60   AIC:                                      831.7
Df Residuals:                      59   BIC:                                      833.8
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
date_delta     9.0516      0.927      9.767      0.000       7.197      10.906
==============================================================================
Omnibus:                       25.458   Durbin-Watson:                   0.027
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                5.652
Skew:                           0.376   Prob(JB):                       0.0592
Kurtosis:                       1.698   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

coef here is 9.0516. Since the question is asking for the slope of this line as seconds per year, the result will be :
9.0516/12 = 0.7543